In [84]:
from transformers import DistilBertTokenizer, DistilBertForTokenClassification, AutoTokenizer, DataCollatorForTokenClassification, Trainer, TrainingArguments

from transformers import pipeline
from torch.utils.data import Dataset, DataLoader



In [2]:
# Load pre-trained DistilBERT model and tokenizer
model_name = "distilbert-base-uncased"
model = DistilBertForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream

In [80]:
# Example raw text and entity offsets
data = [
    {
        "text": "Apple is looking to buy a U.K. startup for $1 billion.",
        "annotations": [(0, 5, "ORG"), (26, 30, "LOC"), (43, 53, "MONEY")]
    },
    {
        "text": "Elon Musk founded SpaceX and Tesla.",
        "annotations": [(0, 9, "PERSON"), (18, 24, "ORG"), (29, 34, "ORG")]
    },
        {
        "text": "Apple is looking to buy a U.K. startup for $1 billion.",
        "annotations": [(0, 5, "ORG"), (26, 30, "LOC"), (43, 53, "MONEY")]
    },
    {
        "text": "Elon Musk founded SpaceX and Tesla.",
        "annotations": [(0, 9, "PERSON"), (18, 24, "ORG"), (29, 34, "ORG")]
    }
]




In [95]:



class NERDataset(Dataset):
    def __init__(self, data_json, tokenizer, label_map) -> None:
        self.data = data_json
        self.tokenizer = tokenizer
        self.label_map = label_map
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data[index]
        text = item['text']
        annots = item['annotations']
        tokenized_text = self.tokenizer(text)
        
        iob_tags = ['O'] * len(tokenized_text.tokens())
        
        for start_offset, end_offset, label in annots:
            # converting offsets based on char indices to token indices
            start_token_index = tokenized_text.char_to_token(start_offset)
            end_token_index = tokenized_text.char_to_token(end_offset - 1) # exclusive
            
            iob_tags[start_token_index] = f"B-{label}"
            
            for token_index in range(start_token_index + 1, end_token_index + 1):
                iob_tags[token_index] = f"I-{label}"
        
        label_indices = [label_map[label] for label in iob_tags]
            
        return {
            # 'text' : text,
            'attention_mask' : tokenized_text['attention_mask'],
            'input_ids' : tokenized_text['input_ids'],
            'labels' : label_indices
            
            # 'tokens' : tokenized_text.tokens() # for debugging only, can be commented out
        }


def generate_label_map():
    return {
    "O": 0,  # Outside
    "B-ORG": 1,
    "I-ORG": 2,
    "B-LOC": 3,
    "I-LOC": 4,
    "B-PERSON": 5,
    "I-PERSON": 6,
    "B-MONEY": 7,
    "I-MONEY": 8,
}


label_map = generate_label_map()            

ner_dataset = NERDataset(data, tokenizer, label_map)
eval_dataset = NERDataset(data, tokenizer, label_map)



In [96]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True)
dataloader = DataLoader(ner_dataset, batch_size=2, collate_fn=data_collator)


In [97]:
next(iter(dataloader))

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]), 'input_ids': tensor([[  101,  6207,  2003,  2559,  2000,  4965,  1037,  1057,  1012,  1047,
          1012, 22752,  2005,  1002,  1015,  4551,  1012,   102],
        [  101,  3449,  2239, 14163,  6711,  2631,  2686,  2595,  1998, 26060,
          1012,   102,     0,     0,     0,     0,     0,     0]]), 'labels': tensor([[   0,    1,    0,    0,    0,    0,    0,    3,    4,    4,    4,    0,
            0,    7,    8,    8,    0,    0],
        [   0,    5,    6,    6,    6,    0,    1,    2,    0,    1,    0,    0,
         -100, -100, -100, -100, -100, -100]])}

In [100]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./ner_output",
    overwrite_output_dir=True,
    num_train_epochs=30,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    use_mps_device=True
   
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ner_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator)

# Start training
trainer.train()

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.00023971761402208358, 'eval_runtime': 0.0423, 'eval_samples_per_second': 94.664, 'eval_steps_per_second': 23.666, 'epoch': 1.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.00011524301953613758, 'eval_runtime': 0.0429, 'eval_samples_per_second': 93.273, 'eval_steps_per_second': 23.318, 'epoch': 2.0}


...

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 4.413797341840109e-06, 'eval_runtime': 0.0531, 'eval_samples_per_second': 75.31, 'eval_steps_per_second': 18.827, 'epoch': 29.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 4.406911557452986e-06, 'eval_runtime': 0.0504, 'eval_samples_per_second': 79.401, 'eval_steps_per_second': 19.85, 'epoch': 30.0}
{'train_runtime': 7.9281, 'train_samples_per_second': 15.136, 'train_steps_per_second': 7.568, 'train_loss': 5.093542858958244e-05, 'epoch': 30.0}


TrainOutput(global_step=60, training_loss=5.093542858958244e-05, metrics={'train_runtime': 7.9281, 'train_samples_per_second': 15.136, 'train_steps_per_second': 7.568, 'train_loss': 5.093542858958244e-05, 'epoch': 30.0})